# (Toy) Fortran Frontend for Loopy

In [10]:
import loopy as lp

In [3]:
src = """subroutine sparse(rowstarts, colindices, values, m, n, nvals, x, y)
  implicit none

  integer rowstarts(m+1), colindices(nvals)
  real*8 values(nvals)
  real*8 x(n), y(n), rowsum

  integer m, n, rowstart, rowend, length, nvals
  integer i, j

  do i = 1, m
    rowstart = rowstarts(i)
    rowend = rowstarts(i+1)
    length = rowend - rowstart

    rowsum = 0
    do j = 1, length
      rowsum = rowsum + &
        x(colindices(rowstart+j-1))*values(rowstart+j-1)
    end do
    y(i) = rowsum
  end do
end
"""

`lp.parse_fortran(src, filename)`

In [13]:
sparse, = lp.parse_fortran(src, "<notebook>")
print(sparse)

---------------------------------------------------------------------------
KERNEL: sparse
---------------------------------------------------------------------------
ARGUMENTS:
colindices: type: np:dtype('int32'), shape: (nvals), dim_tags: (N0:stride:1) aspace: global
m: ValueArg, type: np:dtype('int32')
n: ValueArg, type: np:dtype('int32')
nvals: ValueArg, type: np:dtype('int32')
rowstarts: type: np:dtype('int32'), shape: (m + 1), dim_tags: (N0:stride:1) aspace: global
values: type: np:dtype('float64'), shape: (nvals), dim_tags: (N0:stride:1) aspace: global
x: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
y: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
---------------------------------------------------------------------------
DOMAINS:
[m] -> { [i] : 0 <= i < m }
  [length] -> { [j] : 0 <= j < length }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i: None
j: None


In [14]:
sparse_t = lp.split_iname(sparse, "i", 128)
sparse_t = lp.tag_inames(sparse_t, {"i_outer": "g.0"})
sparse_t = lp.tag_inames(sparse_t, {"i_inner": "l.0"})
sparse_t = lp.split_iname(sparse_t, "j", 4)
sparse_t = lp.tag_inames(sparse_t, {"j_inner": "unr"})
print(sparse_t)

---------------------------------------------------------------------------
KERNEL: sparse
---------------------------------------------------------------------------
ARGUMENTS:
colindices: type: np:dtype('int32'), shape: (nvals), dim_tags: (N0:stride:1) aspace: global
m: ValueArg, type: np:dtype('int32')
n: ValueArg, type: np:dtype('int32')
nvals: ValueArg, type: np:dtype('int32')
rowstarts: type: np:dtype('int32'), shape: (m + 1), dim_tags: (N0:stride:1) aspace: global
values: type: np:dtype('float64'), shape: (nvals), dim_tags: (N0:stride:1) aspace: global
x: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
y: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
---------------------------------------------------------------------------
DOMAINS:
[m] -> { [i_outer, i_inner] : i_inner >= 0 and -128i_outer <= i_inner <= 127 and i_inner < m - 128i_outer }
  [length] -> { [j_outer, j_inner] : j_inner >= 0 and -4j_outer <= j_inner <= 3

Now generate code:

In [15]:
print(lp.generate_code_v2(sparse_t).device_code())

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif
#define LOOPY_CALL_WITH_INTEGER_TYPES(MACRO_NAME) \
    MACRO_NAME(int8, char) \
    MACRO_NAME(int16, short) \
    MACRO_NAME(int32, int) \
    MACRO_NAME(int64, long)
#define LOOPY_DEFINE_FLOOR_DIV_POS_B(SUFFIX, TYPE) \
    inline TYPE loopy_floor_div_pos_b_##SUFFIX(TYPE a, TYPE b) \
    { \
        if (a<0) \
            a = a - (b-1); \
        return a/b; \
    }
LOOPY_CALL_WITH_INTEGER_TYPES(LOOPY_DEFINE_FLOOR_DIV_POS_B)
#undef LOOPY_DEFINE_FLOOR_DIV_POS_B
#undef LOOPY_CALL_WITH_INTEGER_TYPES

__kernel void __attribute__ ((reqd_work_group_size(128, 1, 1))) sparse(__global int const *__restrict__ rowstarts, __global int const *__restrict__ colindices, __global double const *__restrict__ values, int const m, int const n, int const nvals, __global double const *__restrict__ x, __global double *__restrict__ y)
{
  int length;